## Experiment to see if we can recover complete program from .pyc file
Scenario - "base.py" contains secret keys, which are imported in "sample.py". Now, this sample.py is converted to sample.pyc and we think that since nobody can read this our keys are safe. Our objective is to see if we can recover secret keys.

In [1]:
import subprocess as sp
import py_compile


In [2]:
# convert "sample.py" to "sample.pyc"
py_compile.compile("./sample.py")

'.\\__pycache__\\sample.cpython-37.pyc'

In [3]:
# above command gave "sample.cpython-37.pyc" file
# lets rename it to something simpler
sp.run(["move", ".\\__pycache__\\sample.cpython-37.pyc", ".\\sam.pyc"], shell=True)

CompletedProcess(args=['move', '.\\__pycache__\\sample.cpython-37.pyc', '.\\sam.pyc'], returncode=0)

In [4]:
# import function from sam
# lets assume for now that we know which vars are holding keys
from sam import add, KEY1, KEY2, KEY3

print(add(3, 5))
print(KEY1, KEY2, KEY3)  # well we are DOOMED

8
0100a75786009ef8 fd219a1f549cc3db 848c3a5a81b87c14


In [5]:
# lets assume now that we DONT know which vars are holding keys
# peeking inside "sam.pyc"
with open("sam.pyc") as f:
    content = f.read()
print(content)

B

    ÚwÝ^    ã               @   s<   d dl mZmZ dZeeedœdd„Zedkr8eeeƒ dS )é    )ÚKEY1ÚKEY2Z848c3a5a81b87c14)ÚaÚbÚreturnc             C   s   | | S )N© )r   r   r   r   ú./sample.pyÚadd   s    r	   Ú__main__N)Úbaser   r   ZKEY3Úintr	   Ú__name__Úprintr   r   r   r   Ú<module>   s   


In [6]:
# decompiling pyc to recover source code
op = sp.run(["uncompyle6", "sam.pyc"], shell=True, capture_output=True)
print(op)

CompletedProcess(args=['uncompyle6', 'sam.pyc'], returncode=0, stdout=b"# uncompyle6 version 3.7.0\r\n# Python bytecode 3.7 (3394)\r\n# Decompiled from: Python 3.7.7 (default, May  6 2020, 11:45:54) [MSC v.1916 64 bit (AMD64)]\r\n# Embedded file name: ./sample.py\r\n# Compiled at: 2020-06-08 04:57:22\r\n# Size of source mod 2**32: 160 bytes\r\nfrom base import KEY1, KEY2\r\nKEY3 = '848c3a5a81b87c14'\r\n\r\ndef add(a: int, b: int) -> int:\r\n    return a + b\r\n\r\n\r\nif __name__ == '__main__':\r\n    print(KEY1, KEY2)\r\n# okay decompiling sam.pyc\r\n", stderr=b'')


In [9]:
print(op.stdout.decode('utf-8'))

# uncompyle6 version 3.7.0
# Python bytecode 3.7 (3394)
# Decompiled from: Python 3.7.7 (default, May  6 2020, 11:45:54) [MSC v.1916 64 bit (AMD64)]
# Embedded file name: ./sample.py
# Compiled at: 2020-06-08 04:57:22
# Size of source mod 2**32: 160 bytes
from base import KEY1, KEY2
KEY3 = '848c3a5a81b87c14'

def add(a: int, b: int) -> int:
    return a + b


if __name__ == '__main__':
    print(KEY1, KEY2)
# okay decompiling sam.pyc



In [ ]:
# now that the source code is recovered, we can do
print(KEY1, KEY2, KEY3)  # well we are DOOMED

### Conclusion-
saving any important info in pyc file is not safe